# Project

In this Project, you will bring together many of the tools and techniques that you have learned throughout this course into a final project. You can choose from many different paths to get to the solution. 

### Business scenario

You work for a training organization that recently developed an introductory course about machine learning (ML). The course includes more than 40 videos that cover a broad range of ML topics. You have been asked to create an application that will students can use to quickly locate and view video content by searching for topics and key phrases.

You have downloaded all of the videos to an Amazon Simple Storage Service (Amazon S3) bucket. Your assignment is to produce a dashboard that meets your supervisor’s requirements.

## Project steps

To complete this project, you will follow these steps:

1. [Viewing the video files](#1.-Viewing-the-video-files)
2. [Transcribing the videos](#2.-Transcribing-the-videos)
3. [Normalizing the text](#3.-Normalizing-the-text)
4. [Extracting key phrases and topics](#4.-Extracting-key-phrases-and-topics)
5. [Creating the dashboard](#5.-Creating-the-dashboard)

## Useful information

The following cell contains some information that might be useful as you complete this project.

In [1]:
bucket = "c56161a939430l3396553t1w744137092661-labbucket-rn642jaq01e9"
job_data_access_role = 'arn:aws:iam::744137092661:role/service-role/c56161a939430l3396553t1w7-ComprehendDataAccessRole-1P24MSS91ADHP'

## 1. Viewing the video files
([Go to top](#Capstone-8:-Bringing-It-All-Together))


The source video files are located in the following shared Amazon Simple Storage Service (Amazon S3) bucket.

In [2]:
!aws s3 ls s3://aws-tc-largeobjects/CUR-TF-200-ACMNLP-1/video/

2021-04-26 20:17:33  410925369 Mod01_Course Overview.mp4
2021-04-26 20:10:02   39576695 Mod02_Intro.mp4
2021-04-26 20:31:23  302994828 Mod02_Sect01.mp4
2021-04-26 20:17:33  416563881 Mod02_Sect02.mp4
2021-04-26 20:17:33  318685583 Mod02_Sect03.mp4
2021-04-26 20:17:33  255877251 Mod02_Sect04.mp4
2021-04-26 20:23:51   99988046 Mod02_Sect05.mp4
2021-04-26 20:24:54   50700224 Mod02_WrapUp.mp4
2021-04-26 20:26:27   60627667 Mod03_Intro.mp4
2021-04-26 20:26:28  272229844 Mod03_Sect01.mp4
2021-04-26 20:27:06  309127124 Mod03_Sect02_part1.mp4
2021-04-26 20:27:06  195635527 Mod03_Sect02_part2.mp4
2021-04-26 20:28:03  123924818 Mod03_Sect02_part3.mp4
2021-04-26 20:31:28  171681915 Mod03_Sect03_part1.mp4
2021-04-26 20:32:07  285200083 Mod03_Sect03_part2.mp4
2021-04-26 20:33:17  105470345 Mod03_Sect03_part3.mp4
2021-04-26 20:35:10  157185651 Mod03_Sect04_part1.mp4
2021-04-26 20:36:27  187435635 Mod03_Sect04_part2.mp4
2021-04-26 20:36:40  280720369 Mod03_Sect04_part3.mp4
2021-04-26 20:40:01  443479

In [1]:
!pip install moviepy torch transformers accelerate nltk


In [2]:
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from moviepy.editor import VideoFileClip
import boto3
import shutil
import re
from unicodedata import normalize
import nltk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
import ipywidgets as widgets
from IPython.display import display, clear_output, Video
import boto3

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5033:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2501:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: N

## 2. Transcribing the videos
 ([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to implement your solution to transcribe the videos. 

In [3]:

def download_from_s3(s3, bucket_name, key, output_path):
    s3.download_file(bucket_name, key, output_path)

# Following code convert into the audio format
def convert_video_to_audio(video_route, audio_route):
   
    video = VideoFileClip(video_route)
    audio = video.audio
    audio.write_audiofile(audio_route)

# Following code will be processing the video
def process_video(video_index, output_directory):
    
    # Following code will download the video and than send it to convert into the audio format
    print("Downloading video:", video_index)
    video_route = os.path.join(output_directory, 'video.mp4')
    audio_file_name = os.path.splitext(os.path.basename(video_index))[0] + '.mp3'
    print(audio_file_name)
    audio_route = os.path.join(output_directory, audio_file_name)  

    download_from_s3(s3, bucket_name, video_index, video_route)

    print("Converting video to audio:", video_index)
    
    convert_video_to_audio(video_route, audio_route)
    print(f"audio_output/{audio_file_name}")
    result = pipelin(f"audio_output/{audio_file_name}")
    print("Video:", video_index)
    print("Text:", result["text"])
    if not os.path.exists("text_data"):
        os.makedirs("text_data")
    text_file_name = audio_file_name.split(".")
    with open(f"text_data/{text_file_name[0]}.txt","w",encoding="UTF-8") as file:
        file.write(result["text"])

    # remove the all temporary file
    os.remove(video_route)
    os.remove(audio_route)

# AWS S3 configuration
bucket_name = "aws-tc-largeobjects"
output_prefix = "CUR-TF-200-ACMNLP-1/video/"
output_directory = "audio_output"  


if not os.path.exists(output_directory):
    os.makedirs(output_directory)

s3 = boto3.client('s3')

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processors = AutoProcessor.from_pretrained(model_id)
pipelin = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processors.tokenizer,
    feature_extractor=processors.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

# print the list of videos in the S3 bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=output_prefix)
video_indexs = [obj['Key'] for obj in response.get('Contents', [])]

for i, video_index in enumerate(video_indexs):
    print(f"Processing video {i+1}/{len(video_indexs)}")
    process_video(video_index, output_directory)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Processing video 1/46
Mod01_Course Overview.mp3
Converting video to audio: CUR-TF-200-ACMNLP-1/video/Mod01_Course Overview.mp4
MoviePy - Writing audio in audio_output/Mod01_Course Overview.mp3


MoviePy - Done.
audio_output/Mod01_Course Overview.mp3


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Video: CUR-TF-200-ACMNLP-1/video/Mod01_Course Overview.mp4
Text:  Hi, and welcome to Amazon Academy Machine Learning Foundations. In this module, you'll learn about the course objectives, various job roles in the machine learning domain, and where you can go to learn more about machine learning. After completing this module, you should be able to identify course prerequisites and objectives, indicate the role of the data scientist in business, and identify resources for further learning. We're now going to look at the prerequisites for taking this course. Before you take this course, we recommend that you first complete AWS Academy Cloud Foundations. You should also have some general technical knowledge of IT, including foundational computer literacy skills like basic computer concepts, email, file management, and a good understanding of the Internet. We also recommend that you have intermediate skills with Python programming and a general knowledge of applied statistics. Finally, gene

...

MoviePy - Done.
audio_output/Mod06_Intro.mp3
Video: CUR-TF-200-ACMNLP-1/video/Mod06_Intro.mp4
Text:  Introduction to Natural Language Processing Hi, and welcome to Module 6 of AWS Academy Machine Learning, Introduction to Natural Language Processing. In this module, we'll introduce Natural Language Processing, which is also known as NLP. This section includes a description of the major challenges faced by NLP and the overall development process for NLP applications. We'll then review five AWS services you can use to speed up the development of NLP-based applications. After completing this module, you should be able to describe the NLP use cases that are solved by using managed Amazon ML services, and describe the managed Amazon ML services available for NLP. Let's get started.
Processing video 43/46
Mod06_Sect01.mp3
Converting video to audio: CUR-TF-200-ACMNLP-1/video/Mod06_Sect01.mp4
MoviePy - Writing audio in audio_output/Mod06_Sect01.mp3


MoviePy - Done.
audio_output/Mod06_Sect01.mp3
Video: CUR-TF-200-ACMNLP-1/video/Mod06_Sect01.mp4
Text:  We'll get started by reviewing what Natural Language Processing means. Natural Language Processing is also known as NLP. Before we explain what NLP is, we'll consider an example of NLP, Amazon Alexa. Alexa works by having a device, such as an Amazon Echo, record your words. The recording of your speech is sent to Amazon's servers to be analyzed more efficiently. Amazon breaks down your phrase into individual sounds. Then, it connects to a database containing the pronunciation of various words to find which words most closely correspond to the combination of individual sounds. Amazon identifies important words to make sense of the tasks and carry out corresponding functions. For instance, if Alexa notices words like outside or temperature, it will open the weather Alexa skill. Amazon servers then send the information back to your device and Alexa skill. Amazon servers then send the inf

MoviePy - Done.
audio_output/Mod06_Sect02.mp3
Video: CUR-TF-200-ACMNLP-1/video/Mod06_Sect02.mp4
Text:  Welcome back. In this section, we'll review five managed machine learning services you can use for various use cases. These services simplify the process of creating a machine learning application. We'll start by looking at Amazon Transcribe. You can use Amazon Transcribe to recognize speech in audio files and produce a transcription. It can recognize specific voices in an audio file, and you can create a customized vocabulary for terms that are specialized for a particular domain. You can also add a transcription service to your applications by integrating with WebSockets, a transcription service to your applications by integrating with WebSockets, an internet protocol you can use for two-way communication between an application and Amazon Transcribe. Here are some of the more common use cases for Amazon Transcribe. First, medical professionals can record their notes, and Amazon Tran

MoviePy - Done.
audio_output/Mod06_WrapUp.mp3
Video: CUR-TF-200-ACMNLP-1/video/Mod06_WrapUp.mp4
Text:  Welcome back. It's now time to review the module and wrap it up. In summary, in this module, you learned how to describe the NLP use cases that are solved by using managed Amazon ML services and describe the managed ML services available for NLP. Good job. Thanks for watching. We'll see you in the next module. and describe the managed ML services available for NLP. Good job. Thanks for watching. We'll see you in the next module.
Processing video 46/46
Mod07_Sect01.mp3
Converting video to audio: CUR-TF-200-ACMNLP-1/video/Mod07_Sect01.mp4
MoviePy - Writing audio in audio_output/Mod07_Sect01.mp3


MoviePy - Done.
audio_output/Mod07_Sect01.mp3
Video: CUR-TF-200-ACMNLP-1/video/Mod07_Sect01.mp4
Text:  Welcome to Module 7, Course Wrap-Up. Congratulations on completing the AWS Academy Machine Learning course. We'll take a few minutes to review what you've learned and where you can go from here. We're going to start with a review of what you've learned in this course. You learned how to describe machine learning, implement a machine learning pipeline, and use Amazon machine learning services for forecasting, computer vision, and natural language processing. Well done. Although this course isn't designed to prepare you to become certified for the AWS Certified Machine Learning specialty, we'll review how you can continue to work towards that certification. AWS Certification helps you build credibility and confidence by validating your cloud expertise with an industry-recognized credential. It also helps organizations identify skilled professionals who can lead cloud initiatives by usin

## 3. Normalizing the text
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to perform any text normalization steps that are necessary for your solution.

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

def normalize_text(text):
    normalized_text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    normalized_text = normalized_text.lower()
    words = word_tokenize(normalized_text)
    words = [word for word in words if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    normalized_text = ' '.join(words)
    return normalized_text

def normalization_of_files(folder_path):
    normalized_texts = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                normaalize_text = nomalization_of_text(text)
                normalized_texts[filename] = normaalize_text
    return normalized_texts

if not os.path.exists("normalized_text"):
    os.makedirs("normalized_text")
    
folder_path = 'text_data/'  
normalized_texts = normalization_of_files(folder_path)
for filename, normaalize_text in normalized_texts.items():
    print("File:", filename)
    print("\n")
    print("Normalized Text:", normaalize_text)
    print("\n\n")
    with open(os.path.join("normalized_text", filename), 'w', encoding='utf-8') as file:
        file.write(normaalize_text)
            

File: Mod04_Sect02_part2.txt


Normalized Text: hi, welcome back. well continue exploring wrangling time series data. seasonality in data is any kind of repeating observation where the frequency of the observation is stable. for example, in sales, you typically see higher sales at the end of a quarter and into the fourth quarter. consumer retail sees even higher sales in the fourth quarter. be aware that data can have multiple types of seasonality in the same data set. there are many times when you should incorporate seasonality information into your forecast. for instance, localized holidays are a good example for sales. the chart shows that the total revenue generated by arcades has a strong correlation with the number of computer science doctorates awarded in the us. but correlations do not mean causation. if you disagree, see the source for the chart. there are many other correlations plotted on the site, and none of them make any sense. with your own data, be careful that youre no

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 4. Extracting key phrases and topics
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to extract the key phrases and topics from the videos.

In [12]:

def download_stopwords():
    nltk.download('stopwords')

def extract_key_phrases(text):
    # Tokenization of the text data 
    words = word_tokenize(text)

    # Removing all the stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    phrase_one = BigramCollocationFinder.from_words(filtered_words)
    phrase_two = TrigramCollocationFinder.from_words(filtered_words)

    phrase_one_score = phrase_one.score_ngrams(BigramAssocMeasures.likelihood_ratio)
    phrase_two_score = phrase_two.score_ngrams(TrigramAssocMeasures.likelihood_ratio)


    best_phrases_one_list = [phrases_one_list for phrases_one_list, score in sorted(phrase_one_score, key=lambda x: -x[1])[:7]]
    best_phrases_two_list = [phrases_two_list for phrases_two_list, score in sorted(phrase_two_score, key=lambda x: -x[1])[:7]]

    topics_list = []
    for ngram, score in sorted(phrase_one_score + phrase_two_score, key=lambda x: -x[1])[:7]:
        topics_list.append(' '.join(ngram))

    return best_phrases_one_list, best_phrases_two_list, topics_list

def process_normalized_files(input_folder, output_folder):
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.txt'):
            with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as file:
                text = file.read()
                
                key_bigrams, key_trigrams, topics_list = extract_key_phrases(text)
                
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_key_phrases.txt")
                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    output_file.write("Key Phrases:\n")
                    for phrases_one_list in key_bigrams:
                        output_file.write(' '.join(phrases_one_list) + '\n')
                    for phrases_two_list in key_trigrams:
                        output_file.write(' '.join(phrases_two_list) + '\n')
                    output_file.write("\nKey Topics:\n")
                    for topic in topics_list:
                        output_file.write(topic + '\n')
                
                print(f"Processed file: {filename}")
                print("Key Phrases:")
                for phrases_one_list in key_bigrams:
                    print(' '.join(phrases_one_list))
                for phrases_two_list in key_trigrams:
                    print(' '.join(phrases_two_list))
                print("\nKey Topics:")
                for topic in topics_list:
                    print(topic)
                print()


normalized_text_folder = 'normalized_text/'  
output_folder = 'key_phrases/'  
process_normalized_files(normalized_text_folder, output_folder)


Processed file: Mod04_Sect02_part2.txt
Key Phrases:
time series
amazon forecast
appendix excellent
arent independent
want determine
fourth quarter
noise separate
time series data
excellent time series
time series support
trend time series
face time series
random time series
wrangling time series

Key Topics:
time series data
excellent time series
time series support
trend time series
face time series
random time series
wrangling time series

Processed file: Mod05_Sect03_part4_ver2.txt
Key Phrases:
custom labels
amazon recognition
bounding box
bounding boxes
false positives
models calculated
calculated threshold
custom labels operation
detect custom labels
recognition custom labels
custom labels returned
custom labels accuracy
number custom labels
custom labels console

Key Topics:
custom labels operation
detect custom labels
recognition custom labels
custom labels returned
custom labels accuracy
number custom labels
custom labels console

Processed file: Mod05_Intro.txt
Key Phrases:
co

## 5. Creating the dashboard
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to create the dashboard for your solution.

In [16]:
s3_bucket = 'aws-tc-largeobjects'
s3_prefix = 'CUR-TF-200-ACMNLP-1/video/'

search_field = widgets.Text(placeholder='Enter key phrase and topics', description='Search:', layout=widgets.Layout(width='50%'))
search_button = widgets.Button(description='Search', layout=widgets.Layout(margin='0 10px 0 0'))
clear_button = widgets.Button(description='Clear', layout=widgets.Layout(margin='0 0 0 10px'))
output_box = widgets.Output()
video_display = widgets.Output(layout=widgets.Layout(width='70%', height='400px'))

search_box = widgets.HBox([search_field, search_button, clear_button], layout=widgets.Layout(justify_content='flex-start'))

display(search_box, output_box, video_display)

def search_function(button):
    search_term = search_field.value.lower()
    
    matching_files = []
    for filename in os.listdir('key_phrases/'):
        with open(os.path.join('key_phrases/', filename), 'r', encoding='utf-8') as file:
            key_phrases = file.read().lower()
            if search_term in key_phrases:
                matching_files.append(filename[:-16])
    
    with output_box:
        clear_output(wait=True) 
        if matching_files:
            print("Matching files:")
            for file in matching_files:
                matching_file_button = widgets.Button(description=file, layout=widgets.Layout(width='auto'))
                matching_file_button.on_click(lambda x, file=file: display_video(file))
                display(matching_file_button)
        else:
            print("No matching files found.")

def display_video(file_name):
    with video_display:
        clear_output(wait=True)
    
    s3 = boto3.client('s3')
    video_url = f'https://{s3_bucket}.s3.amazonaws.com/{s3_prefix}{file_name}.mp4'
    with video_display:
        display(Video(video_url, width=800)) 

def clear_video(button):
    with video_display:
        clear_output(wait=True)

search_button.on_click(search_function)
clear_button.on_click(clear_video)


Output()

Output(layout=Layout(height='400px', width='70%'))